<a href="https://colab.research.google.com/github/Aida73/Hephia-Project-Group3/blob/main/TestLLM/meditronGpt3/fine_tuning_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers

In [2]:
!pip install accelerate -U
!pip install transformers[torch]

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load your dataset
dataset = load_dataset("pubmed_qa", "pqa_artificial")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
print(dataset["train"][0])

{'pubid': 25429730, 'question': 'Are group 2 innate lymphoid cells ( ILC2s ) increased in chronic rhinosinusitis with nasal polyps or eosinophilia?', 'context': {'contexts': ['Chronic rhinosinusitis (CRS) is a heterogeneous disease with an uncertain pathogenesis. Group 2 innate lymphoid cells (ILC2s) represent a recently discovered cell population which has been implicated in driving Th2 inflammation in CRS; however, their relationship with clinical disease characteristics has yet to be investigated.', 'The aim of this study was to identify ILC2s in sinus mucosa in patients with CRS and controls and compare ILC2s across characteristics of disease.', 'A cross-sectional study of patients with CRS undergoing endoscopic sinus surgery was conducted. Sinus mucosal biopsies were obtained during surgery and control tissue from patients undergoing pituitary tumour resection through transphenoidal approach. ILC2s were identified as CD45(+) Lin(-) CD127(+) CD4(-) CD8(-) CRTH2(CD294)(+) CD161(+) c

In [5]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['question'], max_length=512, truncation=True)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
import torch
torch.cuda.empty_cache()

In [7]:
#data prep
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)
tokenizer.pad_token = tokenizer.eos_token

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=40,
    gradient_accumulation_steps=12,
    learning_rate=1e-3,
    save_steps=10_000,
    save_total_limit=2,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=440, training_loss=3.415186379172585, metrics={'train_runtime': 2268.4061, 'train_samples_per_second': 93.135, 'train_steps_per_second': 0.194, 'total_flos': 4665634606080000.0, 'train_loss': 3.415186379172585, 'epoch': 1.0})

In [8]:
model_path = "./gpt2-finetuned"
tokenizer_path = "./gpt2-finetuned-tokenizer"

# Save the model
model.save_pretrained(model_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_path)

('./gpt2-finetuned-tokenizer/tokenizer_config.json',
 './gpt2-finetuned-tokenizer/special_tokens_map.json',
 './gpt2-finetuned-tokenizer/vocab.json',
 './gpt2-finetuned-tokenizer/merges.txt',
 './gpt2-finetuned-tokenizer/added_tokens.json')